In [41]:
# imports
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from datetime import date
import undetected_chromedriver as uc
from urllib.parse import urlencode, urljoin 
import json



In [42]:
def indeed_scrape(query = "junior software developer", pages=1, wait=5):
    home = 'https://www.indeed.com'
    base = "https://www.indeed.com/jobs?"
    params = {}
    params['q']=query
    params['sort']='date'

    titles = []
    companies = []
    links = []
    
    cities = []
    states= []
    countries=[]
    zips = []

    dates=[]
    mins=[]
    maxes=[]
    types = []
    descriptions = []

    # web driver setup
    driver = uc.Chrome()

    for page in range(0,pages):
        params['start'] = str(page*10)
        url = base + urlencode(params)
        driver.get(url)
        sleep(wait)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        
        jobs_list=soup.find('div', attrs={'id': 'mosaic-provider-jobcards'})

        # extracting data from each job card
        titles += [title.text for title in jobs_list.find_all('h2', class_='jobTitle')] 
        
        companies += [company.text for company in jobs_list.find_all('span',class_='companyName') ]
        
        page_links =[urljoin(home,link['href']) for link in jobs_list.find_all('a', class_='jcs-JobTitle') ]
        links += page_links

        for link in page_links:
            
            driver.get(link)
            sleep(5)
          
            html = driver.page_source
            cup = BeautifulSoup(html, 'html.parser')

            try:
                description = cup.find('div', id='jobDescriptionText')
                descriptions.append(description.text)
            except:
                descriptions.append('')

            try:
                job_script=cup.find('script', attrs={'type':"application/ld+json"})
                job_json=json.loads(str(job_script.contents[0]))
                dates.append(job_json['datePosted'])
            except:
                dates.append('')
            
            try: 
                address = job_json['jobLocation']['address']
                cities.append(address.get('addressLocality'))
                states.append(address.get('addressRegion1'))
                zips.append(address.get('postalCode'))


            except:
                cities.append(None)
                states.append(None)
                zips.append(None)


            try:
                mins.append(int(job_json['baseSalary']['value']['minValue']))
                maxes.append(int(job_json['baseSalary']['value']['maxValue']))
                types.append(job_json['baseSalary']['value']['unitText'])
            except:
                mins.append(None)
                maxes.append(None)
                types.append(None)

    df = pd.DataFrame()
    df['Job_Title'] = titles
    df['Company'] = companies
    df['Link'] = links
    df['Description'] = descriptions
    df['Date Posted'] = dates
    df['Min Salary'] = mins
    df['Max Salary'] = maxes
    df['Salary Type'] = types

    return df
    
        

        


In [43]:
df=indeed_scrape(pages=5)

In [47]:
df

,Job_Title,Company,Link,Description,Date Posted,Min Salary,Max Salary,Salary Type
0,Junior Level PHP Developer,American Partner Solutions,https://www.indeed.com/rc/clk?jk=b662d6673bafa...,\n\nIf you are a Jr. PHP Developer with experi...,"Mon, 13 Mar 2023 16:04:39 GMT",NaN,NaN,None
1,Junior Software Developer,Parsons,https://www.indeed.com/rc/clk?jk=0990abed927a2...,"\n In a world of possibilities, pursue one wit...","Mon, 13 Mar 2023 14:36:15 GMT",59300.0,142600.0,YEAR
2,Junior Software Engineer,Audigent,https://www.indeed.com/rc/clk?jk=c204437d9ac91...,\nAbout Audigent\n Audigent is the leading dat...,"Mon, 13 Mar 2023 16:48:22 GMT",NaN,NaN,None
3,Junior Software Engineer,Urban League,https://www.indeed.com/rc/clk?jk=3434d6e93b013...,"Urban League is looking for a passionate, expe...","Mon, 13 Mar 2023 12:37:19 GMT",125000.0,145000.0,YEAR
4,Entry - Junior Software Developer,DFW Tech Company,https://www.indeed.com/company/DFW-Tech-Compan...,Responsible with assisting the full stack deve...,"Mon, 13 Mar 2023 16:54:33 GMT",55000.0,65000.0,YEAR
...,...,...,...,...,...,...,...,...
70,Jr. Python Developer,"Antra, Inc","https://www.indeed.com/company/Antra,-Inc/jobs...",3+ years of Python and Shell scripting\n3+ yea...,"Wed, 08 Mar 2023 19:52:39 GMT",45.0,-1.0,HOUR
71,Junior Front End Web Developer,Anderson Business Advisors,https://www.indeed.com/rc/clk?jk=e2f579e941435...,\nAnderson Advisors is a business planning and...,"Thu, 09 Mar 2023 04:33:39 GMT",45000.0,55000.0,YEAR
72,Junior Software Engineer,Leidos,https://www.indeed.com/rc/clk?jk=58e6857b34294...,\nDescription \n\nJob Description:\n Leidos is...,"Wed, 08 Mar 2023 20:19:14 GMT",53300.0,110700.0,YEAR
73,Jr. Java Developer,EDR Technology,https://www.indeed.com/company/EDR-Technology/...,"Jr Java Developer \nLocation: Ashburn, Virgini...","Wed, 08 Mar 2023 22:13:04 GMT",0.0,51000.0,YEAR


In [49]:
df.to_csv(f'indeed_{date.today()}.csv')
df.to_json(f'indeed_{date.today()}.json')